In [1]:
!pip install pycocotools


  Created wheel for pycocotools: filename=pycocotools-2.0.2-cp37-cp37m-linux_x86_64.whl size=272447 sha256=524493c3d8ac1143503b4121e80082969428c1acc25094242bb9e3dfa68314a0
  Stored in directory: /root/.cache/pip/wheels/bc/cf/1b/e95c99c5f9d1648be3f500ca55e7ce55f24818b0f48336adaf
Successfully built pycocotools


In [2]:
from pycocotools.coco import COCO
import os
from glob import glob
import pydicom
import seaborn as sns
import matplotlib
import matplotlib.pyplot as plt
import pandas as pd
from matplotlib.colors import ListedColormap, LinearSegmentedColormap 
import cv2
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import torchvision
from torchvision import transforms
import numpy as np
from tqdm import tqdm_notebook as tqdm
from numpy import transpose
import math
from math import log, pi, sqrt
from functools import partial

import torch
from torch import nn, einsum
import torch.nn.functional as F
import PIL
  
import torch
import torch.nn as nn

## Dataloader

In [3]:
class CocoDataset(Dataset):
    """Low-Dose CT dataset."""

    def __init__(self,instance_path='../input/coco-2017-dataset/coco2017/annotations/instances_train2017.json',
                 img_path='../input/coco-2017-dataset/coco2017/train2017',
                 transform=None,):
        """
        Args:
            root_dir (string): Directory with all the images.
            transform (callable, optional): Optional transform to be applied on a sample.
        """
        self.img_paths = img_path
        self.coco=COCO(instance_path)
        self.catIDs = self.coco.getCatIds()
        # print(catIDs)
        imgIds = []
        for x in self.catIDs:
            imgIds += self.coco.getImgIds(catIds = [x])
        self.img_id = imgIds
        self.len = len(imgIds)
        print(self.len)
        
        if transform:
            self.transform = transform
        else:
            self.transform = transforms.Compose([
#                 transforms.Rescale(255),
                transforms.ToTensor()
            ])

    def __len__(self):
        return self.len

    def __getitem__(self, idx):
        if torch.is_tensor(idx):
            idx = idx.tolist()
        img = self.coco.loadImgs(self.img_id[idx])[0]
        image= plt.imread('{}/{}'.format(self.img_paths, img['file_name']))/255.0
        mask = np.zeros((img['height'],img['width']))
        annIds = self.coco.getAnnIds(imgIds=img['id'], catIds=self.catIDs, iscrowd=None)
        anns = self.coco.loadAnns(annIds)
        for i in range(len(anns)):
            mask = np.maximum(self.coco.annToMask(anns[i]), mask)
        mask = cv2.resize(mask, (64,64))
        image = cv2.resize(image, (64,64)).astype(np.float32)
        if(len(image.shape)==2):
            image = cv2.merge([image, image, image])
        mask = (mask > 0.5).astype(np.float32)
        
        if self.transform:
            image = self.transform(image)
            mask = self.transform(mask)
            
        sample = (image, mask)

        return sample


In [4]:
train_set = CocoDataset()
val_set = CocoDataset('../input/coco-2017-dataset/coco2017/annotations/instances_val2017.json', '../input/coco-2017-dataset/coco2017/val2017')

loading annotations into memory...
Done (t=29.62s)
creating index...
index created!
342996
loading annotations into memory...
Done (t=1.03s)
creating index...
index created!
14631


In [5]:
BATCH_SIZE = 50
WORKERS = 0
SHUFFLE = True
train_loader = DataLoader(train_set,  batch_size=BATCH_SIZE,
                        shuffle=SHUFFLE, num_workers=WORKERS)
val_loader = DataLoader(val_set,  batch_size=BATCH_SIZE,
                        shuffle=SHUFFLE, num_workers=WORKERS)

In [6]:
class DenseLayer(nn.Sequential):
    def __init__(self, in_channels, growth_rate):
        super().__init__()
        self.add_module('norm', nn.BatchNorm2d(in_channels))
        self.add_module('relu', nn.ReLU(True))
        self.add_module('conv', nn.Conv2d(in_channels, growth_rate, kernel_size=3,
                                          stride=1, padding=1, bias=True))
        self.add_module('drop', nn.Dropout2d(0.2))

    def forward(self, x):
        return super().forward(x)


class DenseBlock(nn.Module):
    def __init__(self, in_channels, growth_rate, n_layers, upsample=False):
        super().__init__()
        self.upsample = upsample
        self.layers = nn.ModuleList([DenseLayer(
            in_channels + i*growth_rate, growth_rate)
            for i in range(n_layers)])

    def forward(self, x):
        if self.upsample:
            new_features = []
            #we pass all previous activations into each dense layer normally
            #But we only store each dense layer's output in the new_features array
            for layer in self.layers:
                out = layer(x)
                x = torch.cat([x, out], 1)
                new_features.append(out)
            return torch.cat(new_features,1)
        else:
            for layer in self.layers:
                out = layer(x)
                x = torch.cat([x, out], 1) # 1 = channel axis
            return x


class TransitionDown(nn.Sequential):
    def __init__(self, in_channels):
        super().__init__()
        self.add_module('norm', nn.BatchNorm2d(num_features=in_channels))
        self.add_module('relu', nn.ReLU(inplace=True))
        self.add_module('conv', nn.Conv2d(in_channels, in_channels,
                                          kernel_size=1, stride=1,
                                          padding=0, bias=True))
        self.add_module('drop', nn.Dropout2d(0.2))
        self.add_module('maxpool', nn.MaxPool2d(2))

    def forward(self, x):
        return super().forward(x)


class TransitionUp(nn.Module):
    def __init__(self, in_channels, out_channels):
        super().__init__()
        self.convTrans = nn.ConvTranspose2d(
            in_channels=in_channels, out_channels=out_channels,
            kernel_size=3, stride=2, padding=0, bias=True)

    def forward(self, x, skip):
        out = self.convTrans(x)
        out = center_crop(out, skip.size(2), skip.size(3))
        out = torch.cat([out, skip], 1)
        return out


class Bottleneck(nn.Sequential):
    def __init__(self, in_channels, growth_rate, n_layers):
        super().__init__()
        self.add_module('bottleneck', DenseBlock(
            in_channels, growth_rate, n_layers, upsample=True))

    def forward(self, x):
        return super().forward(x)


def center_crop(layer, max_height, max_width):
    _, _, h, w = layer.size()
    xy1 = (w - max_width) // 2
    xy2 = (h - max_height) // 2
    return layer[:, :, xy2:(xy2 + max_height), xy1:(xy1 + max_width)]

class FCDenseNet(nn.Module):
    def __init__(self, in_channels=3, down_blocks=(5,5,5,5,5),
                 up_blocks=(5,5,5,5,5), bottleneck_layers=5,
                 growth_rate=16, out_chans_first_conv=48, n_classes=1):
        super().__init__()
        self.down_blocks = down_blocks
        self.up_blocks = up_blocks
        cur_channels_count = 0
        skip_connection_channel_counts = []

        ## First Convolution ##

        self.add_module('firstconv', nn.Conv2d(in_channels=in_channels,
                  out_channels=out_chans_first_conv, kernel_size=3,
                  stride=1, padding=1, bias=True))
        cur_channels_count = out_chans_first_conv

        #####################
        # Downsampling path #
        #####################

        self.denseBlocksDown = nn.ModuleList([])
        self.transDownBlocks = nn.ModuleList([])
        for i in range(len(down_blocks)):
            self.denseBlocksDown.append(
                DenseBlock(cur_channels_count, growth_rate, down_blocks[i]))
            cur_channels_count += (growth_rate*down_blocks[i])
            skip_connection_channel_counts.insert(0,cur_channels_count)
            self.transDownBlocks.append(TransitionDown(cur_channels_count))

        #####################
        #     Bottleneck    #
        #####################

        self.add_module('bottleneck',Bottleneck(cur_channels_count,
                                     growth_rate, bottleneck_layers))
        prev_block_channels = growth_rate*bottleneck_layers
        cur_channels_count += prev_block_channels

        #######################
        #   Upsampling path   #
        #######################

        self.transUpBlocks = nn.ModuleList([])
        self.denseBlocksUp = nn.ModuleList([])
        for i in range(len(up_blocks)-1):
            self.transUpBlocks.append(TransitionUp(prev_block_channels, prev_block_channels))
            cur_channels_count = prev_block_channels + skip_connection_channel_counts[i]

            self.denseBlocksUp.append(DenseBlock(
                cur_channels_count, growth_rate, up_blocks[i],
                    upsample=True))
            prev_block_channels = growth_rate*up_blocks[i]
            cur_channels_count += prev_block_channels

        ## Final DenseBlock ##

        self.transUpBlocks.append(TransitionUp(
            prev_block_channels, prev_block_channels))
        cur_channels_count = prev_block_channels + skip_connection_channel_counts[-1]

        self.denseBlocksUp.append(DenseBlock(
            cur_channels_count, growth_rate, up_blocks[-1],
                upsample=False))
        cur_channels_count += growth_rate*up_blocks[-1]

        ## Softmax ##

        self.finalConv = nn.Conv2d(in_channels=cur_channels_count,
               out_channels=n_classes, kernel_size=1, stride=1,
                   padding=0, bias=True)
        self.sig = nn.Sigmoid()
    def forward(self, x):
        out = self.firstconv(x)

        skip_connections = []
        for i in range(len(self.down_blocks)):
            out = self.denseBlocksDown[i](out)
            skip_connections.append(out)
            out = self.transDownBlocks[i](out)

        out = self.bottleneck(out)
        for i in range(len(self.up_blocks)):
            skip = skip_connections.pop()
            out = self.transUpBlocks[i](out, skip)
            out = self.denseBlocksUp[i](out)

        out = self.finalConv(out)
        out = self.sig(out)
        return out


In [7]:
model = FCDenseNet()
if torch.cuda.is_available():
    model = model.cuda()

In [8]:
#Binary cross entropy loss function
criterion = nn.BCELoss()
if torch.cuda.is_available():
    criterion = criterion.cuda()

In [9]:
#Adam Optimizer
optimizer = torch.optim.Adam(model.parameters(), lr=0.0001)

In [10]:
# number of epochs to train the model
n_epochs = 6

for epoch in range(1, n_epochs+1):
    # monitor training loss
    train_loss = 0.0
#     train the model
    bar = tqdm(train_loader)
    for data in bar:
        img, mask = data
        if torch.cuda.is_available():
            img, mask = img.cuda(), mask.cuda()
        # clear the gradients of all optimized variables
        optimizer.zero_grad()
        # forward pass
        outputs = model(img)
        loss = criterion(outputs, mask)
        # backward pass
        loss.backward()
        # optimization step (parameter update)
        optimizer.step()
        # update running training loss
        train_loss += loss.item()
        bar.set_description("Processing %s" % loss.item())
    if epoch%2==0:
        torch.save(model, "./modelComplete"+str(epoch))
    valid_loss = 0.0
    val_bar = tqdm(val_loader)
    for data in val_bar:
        
        img, mask = data
        if torch.cuda.is_available():
            img, mask = img.cuda(), mask.cuda()
        
        target = model(img)
       
        loss = criterion(target,mask)
        valid_loss += loss.item()
        val_bar.set_description("Processing %s" % loss.item())
            
    # print avg training statistics 
    train_loss = train_loss/len(train_loader)
    valid_loss = valid_loss/len(val_loader)
    print('Epoch: {} \tTraining Loss: {:.8f} \tValidation Loss: {:.8f} \t'.format(
        epoch, 
        train_loss,valid_loss
        ))

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:8: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  


  0%|          | 0/6860 [00:00<?, ?it/s]

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:29: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`


  0%|          | 0/293 [00:00<?, ?it/s]

torch.Size([50, 3, 64, 64])
torch.Size([50, 3, 64, 64])
torch.Size([50, 3, 64, 64])
torch.Size([50, 3, 64, 64])
torch.Size([50, 3, 64, 64])
torch.Size([50, 3, 64, 64])
torch.Size([50, 3, 64, 64])
torch.Size([50, 3, 64, 64])
torch.Size([50, 3, 64, 64])
torch.Size([50, 3, 64, 64])
torch.Size([50, 3, 64, 64])
torch.Size([50, 3, 64, 64])
torch.Size([50, 3, 64, 64])
torch.Size([50, 3, 64, 64])
torch.Size([50, 3, 64, 64])
torch.Size([50, 3, 64, 64])
torch.Size([50, 3, 64, 64])
torch.Size([50, 3, 64, 64])
torch.Size([50, 3, 64, 64])
torch.Size([50, 3, 64, 64])
torch.Size([50, 3, 64, 64])
torch.Size([50, 3, 64, 64])
torch.Size([50, 3, 64, 64])
torch.Size([50, 3, 64, 64])
torch.Size([50, 3, 64, 64])
torch.Size([50, 3, 64, 64])
torch.Size([50, 3, 64, 64])
torch.Size([50, 3, 64, 64])
torch.Size([50, 3, 64, 64])
torch.Size([50, 3, 64, 64])
torch.Size([50, 3, 64, 64])
torch.Size([50, 3, 64, 64])
torch.Size([50, 3, 64, 64])
torch.Size([50, 3, 64, 64])
torch.Size([50, 3, 64, 64])
torch.Size([50, 3, 6

TypeError: unsupported operand type(s) for /: 'JpegImageFile' and 'float'